### Import Necessary Libraries 

In [ ]:
#!pip install spacy
#!pip install TextBlob
#!pip install tqdm
#!python -m spacy download en_core_web_sm
#!pip install yake
#!pip install transformers torch
#!python -m spacy download en_core_web_lg
#!pip install bertopic --user
#!pip install hdbscan
#!pip install contextualized-topic-models

In [1]:
import re
import string
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import download_corpora
download_corpora.main()

from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm.notebook import tqdm
import yake
from transformers import BertTokenizer, BertModel
import torch

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

from sklearn.decomposition import NMF
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

import gensim
from gensim import corpora
from gensim.models.phrases import Phrases, Phraser
from tqdm.auto import tqdm
import logging

from gensim.models.coherencemodel import CoherenceModel
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashly\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ashl

Finished.


### Data Loading 

In [3]:
# Function to load the data from the csv file to a dataframe and print the shape.
def load_data_and_print_shape(file_path):
    df = pd.read_csv(file_path)
    print(f'Shape: {df.shape}')
    return df

tt_df = load_data_and_print_shape('ted_talks.csv')

Shape: (4005, 19)


In [4]:
# Display the columns present in the dataframe.
tt_df.columns

Index(['talk_id', 'title', 'speaker_1', 'all_speakers', 'occupations',
       'about_speakers', 'views', 'recorded_date', 'published_date', 'event',
       'native_lang', 'available_lang', 'comments', 'duration', 'topics',
       'related_talks', 'url', 'description', 'transcript'],
      dtype='object')

In [5]:
# Display the first five rows  of the dataframe.
tt_df.head()

,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript
0,1,Averting the climate crisis,Al Gore,{0: 'Al Gore'},{0: ['climate advocate']},{0: 'Nobel Laureate Al Gore focused the world’...,3523392,2006-02-25,2006-06-27,TED2006,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",272.0,977,"['alternative energy', 'cars', 'climate change...","{243: 'New thinking on the climate crisis', 54...",https://www.ted.com/talks/al_gore_averting_the...,With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,{0: 'Hans Rosling'},{0: ['global health expert; data visionary']},"{0: 'In Hans Rosling’s hands, data sings. Glob...",14501685,2006-02-22,2006-06-27,TED2006,en,"['ar', 'az', 'bg', 'bn', 'bs', 'cs', 'da', 'de...",628.0,1190,"['Africa', 'Asia', 'Google', 'demo', 'economic...","{2056: ""Own your body's data"", 2296: 'A visual...",https://www.ted.com/talks/hans_rosling_the_bes...,You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,{0: 'David Pogue'},{0: ['technology columnist']},{0: 'David Pogue is the personal technology co...,1920832,2006-02-24,2006-06-27,TED2006,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",124.0,1286,"['computers', 'entertainment', 'interface desi...","{1725: '10 top time-saving tech tips', 2274: '...",https://www.ted.com/talks/david_pogue_simplici...,New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,{0: 'Majora Carter'},{0: ['activist for environmental justice']},{0: 'Majora Carter redefined the field of envi...,2664069,2006-02-26,2006-06-27,TED2006,en,"['ar', 'bg', 'bn', 'ca', 'cs', 'de', 'en', 'es...",219.0,1116,"['MacArthur grant', 'activism', 'business', 'c...",{1041: '3 stories of local eco-entrepreneurshi...,https://www.ted.com/talks/majora_carter_greeni...,"In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,{0: 'Sir Ken Robinson'},"{0: ['author', 'educator']}","{0: ""Creativity expert Sir Ken Robinson challe...",65051954,2006-02-25,2006-06-27,TED2006,en,"['af', 'ar', 'az', 'be', 'bg', 'bn', 'ca', 'cs...",4931.0,1164,"['children', 'creativity', 'culture', 'dance',...","{865: 'Bring on the learning revolution!', 173...",https://www.ted.com/talks/sir_ken_robinson_do_...,Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


### Data Pre-Processing 

In [6]:
# Function to extract only the columns required for analysis.
def extract_required_columns(df):
    required_columns = ['talk_id', 'title', 'speaker_1', 'occupations', 'event', 
                        'published_date', 'duration', 'topics', 'description', 
                        'views', 'comments', 'transcript']
    return df[required_columns]
tt_df_sub = extract_required_columns(tt_df)

In [7]:
# Function to check for null values and duplicates in the dataframe.
def check_data_quality(df):
    # Check for null values in the DataFrame.
    null_values = df.isnull().sum()
    print("Null values per column:")
    print(null_values)
    print("\n")

    # Check for duplicate values.
    duplicate_rows = df[df.duplicated()]

    # To see the number of duplicate rows.
    num_duplicate_rows = duplicate_rows.shape[0]
    print(f"Number of duplicate rows: {num_duplicate_rows}\n")

    # View the duplicated rows if any.
    if num_duplicate_rows > 0:
        print("Duplicated rows:")
        print(duplicate_rows)
    else:
        print("No duplicated rows found.")

# Use the function:
check_data_quality(tt_df_sub)

Null values per column:
talk_id             0
title               0
speaker_1           0
occupations       522
event               0
published_date      0
duration            0
topics              0
description         0
views               0
comments          655
transcript          0
dtype: int64


Number of duplicate rows: 0

No duplicated rows found.


In [8]:
# Check for data types of columns.
print(tt_df_sub.dtypes)

talk_id             int64
title              object
speaker_1          object
occupations        object
event              object
published_date     object
duration            int64
topics             object
description        object
views               int64
comments          float64
transcript         object
dtype: object


#### Basic Pre-Processing 

In [9]:
# Function for basic data pre-processing.

def preprocess_dataframe(df):
    def fill_missing_values(df):
        """Replace null values with default values."""
        df['occupations'].fillna('Unknown', inplace=True)
        df['comments'].fillna(0, inplace=True)
        return df
    
    def clean_strings(df, columns, chars_to_remove):
        """Remove specified characters from given columns."""
        remove_dict = {ord(char): None for char in chars_to_remove}
        for col in columns:
            df[col] = df[col].str.translate(remove_dict)
        return df

    def remove_numbers_from_column(df, column):
        """Remove numbers from specified column."""
        df[column] = df[column].str.replace(r'\d+', '', regex=True)
        return df

    def convert_duration_to_minutes(df, column):
        """Convert duration from seconds to minutes."""
        df[column] = (df[column] / 60).round(2)
        return df

    def convert_to_datetime(df, column):
        """Convert specified column to datetime."""
        df[column] = pd.to_datetime(df[column])
        return df

    def extract_year(df, source_column, new_column):
        """Extract year from source column and create new column."""
        df[new_column] = pd.to_datetime(df[source_column]).dt.year
        return df

    def convert_column_to_int64(df, column):
        """Convert specified column to int64."""
        df[column] = df[column].astype('int64')
        return df

    def rearrange_columns(df, columns_order):
        """Rearrange dataframe columns based on given order."""
        return df[columns_order]

    def rename_columns(df, mapping):
        """Rename dataframe columns based on given mapping."""
        return df.rename(columns=mapping)

    # Make a copy to avoid SettingWithCopyWarning.
    df = df.copy()

    df = fill_missing_values(df)
    df = clean_strings(df, ['occupations', 'topics'], ['{', '#', ':', '[', '\'', ';', ',', ']', '}'])
    df = remove_numbers_from_column(df, 'occupations')
    df = convert_duration_to_minutes(df, 'duration')
    df = convert_to_datetime(df, 'published_date')
    df = extract_year(df, 'published_date', 'year')
    df = convert_column_to_int64(df, 'comments')
    df = rearrange_columns(df, [df.columns[-1]] + df.columns[:-1].tolist())
    df = rename_columns(df, {'speaker_1': 'speaker'})

    return df

# Main preprocessing function.
tt_df_sub = preprocess_dataframe(tt_df_sub)

In [10]:
# Verify if the null values are handled.
null_values = tt_df_sub.isnull().sum()
print(null_values)

year              0
talk_id           0
title             0
speaker           0
occupations       0
event             0
published_date    0
duration          0
topics            0
description       0
views             0
comments          0
transcript        0
dtype: int64


In [11]:
# Display first five rows of the cleaned dataframe.
tt_df_sub.head()

,year,talk_id,title,speaker,occupations,event,published_date,duration,topics,description,views,comments,transcript
0,2006,1,Averting the climate crisis,Al Gore,climate advocate,TED2006,2006-06-27,16.28,alternative energy cars climate change culture...,With the same humor and humanity he exuded in ...,3523392,272,"Thank you so much, Chris. And it's truly a gre..."
1,2006,92,The best stats you've ever seen,Hans Rosling,global health expert data visionary,TED2006,2006-06-27,19.83,Africa Asia Google demo economics global issue...,You've never seen data presented like this. Wi...,14501685,628,"About 10 years ago, I took on the task to teac..."
2,2006,7,Simplicity sells,David Pogue,technology columnist,TED2006,2006-06-27,21.43,computers entertainment interface design media...,New York Times columnist David Pogue takes aim...,1920832,124,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,2006,53,Greening the ghetto,Majora Carter,activist for environmental justice,TED2006,2006-06-27,18.60,MacArthur grant activism business cities envir...,"In an emotionally charged talk, MacArthur-winn...",2664069,219,If you're here today — and I'm very happy that...
4,2006,66,Do schools kill creativity?,Sir Ken Robinson,author educator,TED2006,2006-06-27,19.40,children creativity culture dance education pa...,Sir Ken Robinson makes an entertaining and pro...,65051954,4931,Good morning. How are you? (Audience) Good. It...


#### Advanced Text Pre-processing

In [12]:
# More Pre-processing.

def preprocess_text(text):
    # Convert text to lowercase.
    text = text.lower()
    
    # Remove text in square brackets & parenthesis.
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\(.*?\)', '', text)

    # Remove punctuation.
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize.
    tokens = word_tokenize(text)

    # Remove stopwords.
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize.
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

# Apply preprocessing to the dataframe columns.
tt_df_sub['description'] = tt_df_sub['description'].apply(preprocess_text)
tt_df_sub['transcript'] = tt_df_sub['transcript'].apply(preprocess_text)

### Feature Engineering 

#### Word Count

In [13]:
# Function to determine the word count of description and transcript columns.
def compute_word_counts(df, column_name):
    return df[column_name].apply(lambda x: len(x.split()))

def add_word_counts(df):
    df['description_word_count'] = compute_word_counts(df, 'description')
    df['transcript_word_count'] = compute_word_counts(df, 'transcript')
    return df

tt_df_sub = add_word_counts(tt_df_sub)

#### TF-IDF

In [14]:
# Function to compute tf-idf.
def compute_tfidf(df, column_name, max_features=5000):
    vectorizer = TfidfVectorizer(max_df=0.50, max_features=max_features, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df[column_name])
    return vectorizer, tfidf_matrix

description_vectorizer, description_tfidf_matrix = compute_tfidf(tt_df_sub, 'description')
transcript_vectorizer, transcript_tfidf_matrix = compute_tfidf(tt_df_sub, 'transcript')

#### N-Grams 

In [15]:
# Function to compute n-grams.
def compute_tfidf_with_ngrams(df, column_name, max_features=5000, ngram_range=(1,2)):
    vectorizer = TfidfVectorizer(max_df=0.80, max_features=max_features, stop_words='english', ngram_range=ngram_range)
    tfidf_matrix = vectorizer.fit_transform(df[column_name])
    return vectorizer, tfidf_matrix

vectorizer, tfidf_matrix = compute_tfidf_with_ngrams(tt_df_sub, 'transcript')

#### Sentiment Scores 

In [ ]:
# Function to compute sentiment scores.
def compute_sentiment(text):
    return TextBlob(text).sentiment.polarity

def add_sentiment_scores(df):
    df['description_sentiment'] = df['description'].apply(compute_sentiment)
    df['transcript_sentiment'] = df['transcript'].apply(compute_sentiment)
    return df

tt_df_sub = add_sentiment_scores(tt_df_sub)

#### Named Entity Recognition 

In [ ]:
#Function to perform Named Entity Recognition.

# Load the large English model.
nlp = spacy.load("en_core_web_lg")

def extract_named_entities(text):
    # Defining the entity labels we are interested in
    relevant_entities = ['PERSON', 'ORG', 'GPE', 'DATE', 'EVENT', 'PRODUCT', 'WORK_OF_ART', 'LAW']
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ in relevant_entities]

def clean_entities(entities):
    return [ent for ent in entities if len(ent) > 2 and ent.lower() not in STOP_WORDS]

def add_named_entities(df, column_name):
    new_column = f"{column_name}_named_entities"
    tqdm.pandas(desc=f"Processing {column_name}")
    df[new_column] = df[column_name].progress_apply(extract_named_entities)
    df[new_column] = df[new_column].apply(clean_entities)
    return df

tt_df_sub = add_named_entities(tt_df_sub, 'description')
tt_df_sub = add_named_entities(tt_df_sub, 'transcript')

#### Keyword Extraction 

In [ ]:
# Function to perform keyword extraction.
def extract_keywords(text):
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]

# Apply the keyword extraction function to the 'transcript' column with tqdm progress.
tt_df_sub['transcript_keywords'] = tqdm(tt_df_sub['transcript'].apply(extract_keywords), desc='Extracting Keywords')

#### Document Embedding using BERT 

In [ ]:
# Load pre-trained BERT model and tokenizer.
#model_name = "bert-base-uncased"
#tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertModel.from_pretrained(model_name)

In [ ]:
# Function to perform BERT embedding.
#def get_bert_embedding(text):
    # Tokenize input text
    #inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Get BERT embeddings.
    #with torch.no_grad():
        #outputs = model(**inputs)
    
    # Use the embedding of the [CLS] token as representation.
    #embedding = outputs.last_hidden_state[:, 0, :].numpy()

    #return embedding

# Apply the function and see progress with tqdm.
#tt_df_sub['bert_embedding'] = [get_bert_embedding(x) for x in tqdm(tt_df_sub['transcript'])]

In [ ]:
# Convert DataFrame to CSV and Json for verification.
tt_df_sub.to_csv('tt_df_sub.csv', index=False)
#tt_df_sub.to_json('tt_df_sub.json', orient='records', lines=True)

### Exploratory Data Analysis 

#### Descriptive Statistics 

In [ ]:
tt_df_sub.describe()

#### Word Count Distribution 

In [ ]:
# Histogram to plot the distribution of description and transcript word count.
plt.figure(figsize=(12, 6))
sns.histplot(tt_df_sub['description_word_count'], kde=True, bins=50)
plt.title('Distribution of Description Word Counts')
plt.show()

plt.figure(figsize=(12, 6))
sns.histplot(tt_df_sub['transcript_word_count'], kde=True, bins=50)
plt.title('Distribution of Transcript Word Counts')
plt.show() 

#### Sentiment Distribution 

In [ ]:
# Histogram to plot the distribution of transcript sentiment.
plt.figure(figsize=(12, 6))
sns.histplot(tt_df_sub['transcript_sentiment'], kde=True, bins=50)
plt.title('Distribution of Transcript Sentiment')
plt.show()

#### Word Cloud

In [ ]:
# Word cloud for transcript.
def generate_wordcloud(text):
    wordcloud = WordCloud(background_color='white', width=800, height=800).generate(text)
    plt.figure(figsize=(8, 8))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

generate_wordcloud(' '.join(tt_df_sub['transcript'].tolist()))

### Model Building 

#### LDA

In [17]:
# Split each transcript into words.
texts = tt_df_sub['transcript'].apply(lambda x: x.split()).tolist()

# Build the bigram and trigram models.
bigram = Phrases(texts, min_count=5, threshold=100)  
trigram = Phrases(bigram[texts], threshold=100)

# Construct bigram and trigram.
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# Define functions for creating bigrams and trigrams with tqdm progress.
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in tqdm(texts, desc='Making bigrams')]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in tqdm(texts, desc='Making trigrams')]

# Form bigrams and trigrams with tqdm progress.
texts_with_bigrams = make_bigrams(texts)
texts_with_trigrams = make_trigrams(texts_with_bigrams)

# Recreate the dictionary to include bigrams and trigrams.
dictionary = corpora.Dictionary(texts_with_trigrams)

# Filter out extremes to remove tokens that appear too frequently or infrequently.
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Convert the documents to a bag-of-words representation with tqdm progress.
corpus = [dictionary.doc2bow(text) for text in tqdm(texts_with_trigrams, desc='Creating corpus')]

# Set up logging for progress tracking.
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train LDA model with progress updates and additional hyperparameters.
NUM_TOPICS = 15
ldamodel = gensim.models.LdaMulticore(
    corpus,
    num_topics=NUM_TOPICS,
    id2word=dictionary,
    passes=30,
    workers=3,
    alpha='asymmetric',  
    eta='auto',        
    random_state=100,   
    per_word_topics=True
)

# Print topics.
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

Making bigrams:   0%|          | 0/4005 [00:00<?, ?it/s]

Making trigrams:   0%|          | 0/4005 [00:00<?, ?it/s]

Creating corpus:   0%|          | 0/4005 [00:00<?, ?it/s]

2023-11-20 14:16:38,064 : INFO : using asymmetric alpha [0.15263604, 0.121313125, 0.10065699, 0.08601167, 0.07508676, 0.06662436, 0.05987621, 0.05436933, 0.04979008, 0.045922287, 0.04261209, 0.039747022, 0.037242956, 0.035035703, 0.03307544]
2023-11-20 14:16:38,064 : INFO : using serial LDA version on this node
2023-11-20 14:16:38,107 : INFO : running online LDA training, 15 topics, 30 passes over the supplied corpus of 4005 documents, updating every 6000 documents, evaluating every ~4005 documents, iterating 50x with a convergence threshold of 0.001000
2023-11-20 14:16:38,112 : INFO : training LDA model using 3 processes
2023-11-20 14:16:41,238 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/4005, outstanding queue size 1
2023-11-20 14:16:41,465 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/4005, outstanding queue size 2
2023-11-20 14:16:41,466 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #4005/4005, outstanding queue

2023-11-20 14:17:04,353 : INFO : topic #1 (0.121): 0.007*"water" + 0.004*"animal" + 0.004*"ocean" + 0.003*"show" + 0.003*"body" + 0.003*"ive" + 0.003*"human" + 0.003*"hand" + 0.003*"weve" + 0.002*"sort"
2023-11-20 14:17:04,355 : INFO : topic #0 (0.153): 0.003*"sleep" + 0.003*"problem" + 0.003*"might" + 0.003*"human" + 0.003*"memory" + 0.003*"person" + 0.002*"water" + 0.002*"body" + 0.002*"study" + 0.002*"found"
2023-11-20 14:17:04,356 : INFO : topic diff=0.240023, rho=0.377897
2023-11-20 14:17:04,396 : INFO : -8.305 per-word bound, 316.3 perplexity estimate based on a held-out corpus of 5 documents with 2034 words
2023-11-20 14:17:04,403 : INFO : PROGRESS: pass 5, dispatched chunk #0 = documents up to #2000/4005, outstanding queue size 1
2023-11-20 14:17:04,638 : INFO : PROGRESS: pass 5, dispatched chunk #1 = documents up to #4000/4005, outstanding queue size 2
2023-11-20 14:17:04,641 : INFO : PROGRESS: pass 5, dispatched chunk #2 = documents up to #4005/4005, outstanding queue size 3


...

2023-11-20 14:18:06,389 : INFO : topic #1 (0.121): 0.011*"water" + 0.006*"animal" + 0.006*"ocean" + 0.004*"foot" + 0.003*"fish" + 0.003*"ive" + 0.003*"body" + 0.003*"big" + 0.003*"show" + 0.003*"long"
2023-11-20 14:18:06,390 : INFO : topic #0 (0.153): 0.006*"god" + 0.005*"sleep" + 0.005*"human" + 0.004*"compassion" + 0.004*"death" + 0.003*"others" + 0.003*"person" + 0.003*"age" + 0.003*"great" + 0.003*"memory"
2023-11-20 14:18:06,393 : INFO : topic diff=0.255231, rho=0.242518
2023-11-20 14:18:06,438 : INFO : -8.212 per-word bound, 296.6 perplexity estimate based on a held-out corpus of 5 documents with 2034 words
2023-11-20 14:18:06,441 : INFO : PROGRESS: pass 15, dispatched chunk #0 = documents up to #2000/4005, outstanding queue size 1
2023-11-20 14:18:06,663 : INFO : PROGRESS: pass 15, dispatched chunk #1 = documents up to #4000/4005, outstanding queue size 2
2023-11-20 14:18:06,668 : INFO : PROGRESS: pass 15, dispatched chunk #2 = documents up to #4005/4005, outstanding queue size 

2023-11-20 14:18:34,154 : INFO : topic #1 (0.121): 0.012*"water" + 0.007*"animal" + 0.007*"ocean" + 0.004*"foot" + 0.004*"fish" + 0.003*"sea" + 0.003*"big" + 0.003*"ive" + 0.003*"specie" + 0.003*"body"
2023-11-20 14:18:34,154 : INFO : topic #0 (0.153): 0.007*"god" + 0.005*"human" + 0.004*"death" + 0.004*"sleep" + 0.004*"compassion" + 0.003*"others" + 0.003*"great" + 0.003*"age" + 0.003*"history" + 0.002*"may"
2023-11-20 14:18:34,154 : INFO : topic diff=0.194044, rho=0.213189
2023-11-20 14:18:34,206 : INFO : -8.205 per-word bound, 295.0 perplexity estimate based on a held-out corpus of 5 documents with 2034 words
2023-11-20 14:18:34,208 : INFO : PROGRESS: pass 20, dispatched chunk #0 = documents up to #2000/4005, outstanding queue size 1
2023-11-20 14:18:34,407 : INFO : PROGRESS: pass 20, dispatched chunk #1 = documents up to #4000/4005, outstanding queue size 2
2023-11-20 14:18:34,419 : INFO : PROGRESS: pass 20, dispatched chunk #2 = documents up to #4005/4005, outstanding queue size 3

2023-11-20 14:19:02,160 : INFO : topic #1 (0.121): 0.012*"water" + 0.008*"animal" + 0.007*"ocean" + 0.004*"fish" + 0.004*"foot" + 0.004*"sea" + 0.003*"specie" + 0.003*"big" + 0.003*"long" + 0.003*"earth"
2023-11-20 14:19:02,160 : INFO : topic #0 (0.153): 0.007*"god" + 0.005*"human" + 0.005*"death" + 0.004*"compassion" + 0.004*"sleep" + 0.003*"history" + 0.003*"great" + 0.003*"others" + 0.003*"age" + 0.003*"culture"
2023-11-20 14:19:02,160 : INFO : topic diff=0.144567, rho=0.192441
2023-11-20 14:19:02,215 : INFO : -8.197 per-word bound, 293.4 perplexity estimate based on a held-out corpus of 5 documents with 2034 words
2023-11-20 14:19:02,218 : INFO : PROGRESS: pass 25, dispatched chunk #0 = documents up to #2000/4005, outstanding queue size 1
2023-11-20 14:19:02,511 : INFO : PROGRESS: pass 25, dispatched chunk #1 = documents up to #4000/4005, outstanding queue size 2
2023-11-20 14:19:02,900 : INFO : PROGRESS: pass 25, dispatched chunk #2 = documents up to #4005/4005, outstanding queue 

2023-11-20 14:19:28,352 : INFO : topic #1 (0.121): 0.013*"water" + 0.008*"animal" + 0.007*"ocean" + 0.005*"fish" + 0.004*"foot" + 0.004*"sea" + 0.004*"specie" + 0.003*"big" + 0.003*"long" + 0.003*"earth"
2023-11-20 14:19:28,353 : INFO : topic #0 (0.153): 0.008*"god" + 0.006*"human" + 0.005*"death" + 0.004*"compassion" + 0.004*"history" + 0.003*"great" + 0.003*"others" + 0.003*"culture" + 0.003*"sleep" + 0.003*"story"
2023-11-20 14:19:28,353 : INFO : topic diff=0.110415, rho=0.176770
2023-11-20 14:19:28,405 : INFO : -8.190 per-word bound, 292.0 perplexity estimate based on a held-out corpus of 5 documents with 2034 words
2023-11-20 14:19:28,430 : INFO : LdaMulticore lifecycle event {'msg': 'trained LdaMulticore<num_terms=9529, num_topics=15, decay=0.5, chunksize=2000> in 170.33s', 'datetime': '2023-11-20T14:19:28.430389', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-11-2

(0, '0.008*"god" + 0.006*"human" + 0.005*"death" + 0.004*"compassion" + 0.004*"history"')
(1, '0.013*"water" + 0.008*"animal" + 0.007*"ocean" + 0.005*"fish" + 0.004*"foot"')
(2, '0.020*"brain" + 0.012*"patient" + 0.008*"disease" + 0.008*"health" + 0.007*"body"')
(3, '0.009*"word" + 0.008*"feel" + 0.008*"language" + 0.005*"love" + 0.005*"person"')
(4, '0.012*"human" + 0.009*"computer" + 0.008*"data" + 0.008*"technology" + 0.007*"machine"')
(5, '0.006*"government" + 0.005*"question" + 0.004*"information" + 0.004*"group" + 0.004*"system"')
(6, '0.017*"woman" + 0.009*"child" + 0.008*"family" + 0.006*"girl" + 0.006*"men"')
(7, '0.007*"kid" + 0.006*"company" + 0.005*"data" + 0.005*"money" + 0.005*"guy"')
(8, '0.029*"city" + 0.015*"building" + 0.009*"design" + 0.008*"space" + 0.008*"car"')
(9, '0.021*"country" + 0.007*"africa" + 0.006*"global" + 0.005*"government" + 0.005*"china"')
(10, '0.012*"food" + 0.008*"energy" + 0.006*"problem" + 0.005*"company" + 0.005*"system"')
(11, '0.013*"story" +

In [18]:
# Coherence Score.
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

2023-11-20 14:19:46,506 : INFO : using ParallelWordOccurrenceAccumulator<processes=11, batch_size=64> to estimate probabilities from sliding windows
2023-11-20 14:19:57,737 : INFO : 1 batches submitted to accumulate stats from 64 documents (83903 virtual)
2023-11-20 14:19:57,747 : INFO : 2 batches submitted to accumulate stats from 128 documents (146504 virtual)
2023-11-20 14:19:57,764 : INFO : 3 batches submitted to accumulate stats from 192 documents (213358 virtual)
2023-11-20 14:19:57,787 : INFO : 4 batches submitted to accumulate stats from 256 documents (278533 virtual)
2023-11-20 14:19:57,814 : INFO : 5 batches submitted to accumulate stats from 320 documents (350213 virtual)
2023-11-20 14:19:57,833 : INFO : 6 batches submitted to accumulate stats from 384 documents (429970 virtual)
2023-11-20 14:19:57,976 : INFO : 7 batches submitted to accumulate stats from 448 documents (487391 virtual)
2023-11-20 14:19:57,986 : INFO : 8 batches submitted to accumulate stats from 512 document


Coherence Score:  0.4145271214519435


#### LSA 

In [25]:
from sklearn.decomposition import TruncatedSVD

# Define the number of topics
NUM_TOPICS = 15

# Step 1: Apply LSA
lsa_model = TruncatedSVD(n_components=NUM_TOPICS, random_state=42)
lsa_model.fit(transcript_tfidf_matrix)

# Step 2: Extract and display topics
feature_names = transcript_vectorizer.get_feature_names_out()
for i, topic in enumerate(lsa_model.components_):
    top_features_ind = topic.argsort()[:-10 - 1:-1]
    top_features = [feature_names[index] for index in top_features_ind]
    weights = topic[top_features_ind]

    print(f"Topic {i+1}:")
    print(", ".join(top_features))
    print("\n")

Topic 1:
woman, story, child, country, brain, city, school, problem, kid, technology


Topic 2:
woman, girl, child, men, school, family, kid, story, country, mother


Topic 3:
brain, cell, cancer, woman, patient, body, disease, neuron, blood, drug


Topic 4:
cell, cancer, health, patient, country, disease, drug, woman, percent, africa


Topic 5:
woman, water, ocean, earth, planet, men, animal, girl, sea, fish


Topic 6:
woman, universe, data, galaxy, star, men, planet, earth, information, theory


Topic 7:
brain, neuron, ocean, child, food, water, fish, language, climate, animal


Topic 8:
city, brain, woman, building, neuron, design, architecture, men, space, urban


Topic 9:
robot, woman, water, ocean, fish, machine, animal, girl, ai, computer


Topic 10:
school, child, kid, brain, universe, student, teacher, education, city, woman


Topic 11:
cell, language, school, dna, student, music, teacher, africa, education, english


Topic 12:
robot, war, ai, country, universe, child, africa,

In [26]:
# Prepare the list of top words for each topic from LSA.
top_n = 10  # Number of top words to consider for each topic.
topics = [[feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]] for topic in lsa_model.components_]

# Prepare the Gensim dictionary and corpus.
dictionary = Dictionary(texts_with_trigrams)
corpus = [dictionary.doc2bow(text) for text in texts_with_trigrams]

# Create the Coherence Model.
coherence_model = CoherenceModel(topics=topics, texts=texts_with_trigrams, dictionary=dictionary, coherence='c_v')

# Compute the Coherence Score.
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

2023-11-20 14:34:32,632 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-11-20 14:34:35,255 : INFO : built Dictionary<76564 unique tokens: ['20', '238', '28second', '30second', '31']...> from 4005 documents (total 3416317 corpus positions)
2023-11-20 14:34:35,255 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<76564 unique tokens: ['20', '238', '28second', '30second', '31']...> from 4005 documents (total 3416317 corpus positions)", 'datetime': '2023-11-20T14:34:35.255151', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-11-20 14:34:37,007 : INFO : using ParallelWordOccurrenceAccumulator<processes=11, batch_size=64> to estimate probabilities from sliding windows
2023-11-20 14:34:49,030 : INFO : 1 batches submitted to accumulate stats from 64 documents (81929 virtual)
2023-11-20 14:34:49,088 : INFO : 2 batches submitted to accumulate stats fr

Coherence Score: 0.4046219173348973


#### Non-Negative Matrix Factorization (NMF)

In [27]:
num_topics = 15  

# Create an NMF model.
nmf_model = NMF(
    n_components=num_topics,        # Number of topics.
    init='nndsvd',                 # Initialization method .
    random_state=42,               # Random seed for reproducibility.
    alpha=0.1,                   # Regularization parameters.
    l1_ratio=0.5,                  # Sparsity parameter. 
    max_iter=200,                  # Maximum number of iterations.
    verbose=1                      # Verbosity level (1 for progress updates).
)

In [28]:
# Fit the model to the 'transcript' TF-IDF matrix.
nmf_model.fit(transcript_tfidf_matrix)

C:\Users\ashly\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


violation: 1.0
violation: 0.3169809045245732
violation: 0.22661446577718866
violation: 0.16499193648219967
violation: 0.11732147148271226
violation: 0.09059244295447151
violation: 0.06972224470310531
violation: 0.05530169672659834
violation: 0.0466509717633364
violation: 0.0388682695211769
violation: 0.03222279074174161
violation: 0.02706648860005983
violation: 0.023221417683279927
violation: 0.020301419964456852
violation: 0.017879498761617568
violation: 0.015990983131277607
violation: 0.014411395882686166
violation: 0.013035246821910004
violation: 0.011761933200442453
violation: 0.010657887009088438
violation: 0.009742087633558575
violation: 0.009005365611653333
violation: 0.008403996067707711
violation: 0.00788386901112701
violation: 0.007374620032212143
violation: 0.006855574624390159
violation: 0.006310407768147964
violation: 0.00568176354771571
violation: 0.005043680007553677
violation: 0.00445555244477551
violation: 0.0039258783454808235
violation: 0.003453206658418631
violation

NMF(alpha=0.1, init='nndsvd', l1_ratio=0.5, n_components=15, random_state=42,
    verbose=1)

In [29]:
# Get the topics and the topic-word distributions.
nmftopics = nmf_model.components_

n_top_words = 10  
feature_names = transcript_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmftopics):
    top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")

Topic #1: story, love, feel, ive, thought, talk, started, went, question, book
Topic #2: water, plant, energy, climate, food, earth, carbon, planet, forest, tree
Topic #3: patient, cancer, health, doctor, disease, drug, care, treatment, medical, hospital
Topic #4: country, africa, government, percent, dollar, economy, china, money, company, business
Topic #5: ocean, fish, sea, coral, reef, animal, shark, marine, fishing, specie
Topic #6: woman, men, girl, sex, gender, female, male, sexual, mother, boy
Topic #7: brain, neuron, memory, sleep, signal, activity, disorder, body, cortex, behavior
Topic #8: city, building, space, design, architecture, urban, street, park, community, neighborhood
Topic #9: school, kid, child, student, teacher, education, girl, parent, classroom, learning
Topic #10: universe, galaxy, star, planet, light, earth, space, telescope, particle, sun
Topic #11: cell, cancer, dna, body, gene, tissue, stem, protein, blood, immune
Topic #12: robot, ai, robotics, machine, 

In [30]:
# Get document-topic matrix.
document_topic_matrix = nmf_model.transform(transcript_tfidf_matrix)

# Assign each document to the most probable topic.
dominant_topic_for_each_doc = document_topic_matrix.argmax(axis=1) + 1  # +1 to start topics from 1

violation: 1.0
violation: 0.152577051651251
violation: 0.008614368778717537
violation: 0.0009801830371258743
violation: 0.00011631146678459456
violation: 1.031063061798322e-05
Converged at iteration 7


C:\Users\ashly\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [31]:
# Create a Gensim Dictionary.
transcript_dict = Dictionary(tt_df_sub['transcript'].apply(str.split))

# Convert the NMF topics into a format suitable for coherence calculation.
nmf_topics = [[feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]] for topic in nmftopics]

# Calculate the coherence score using the 'c_v' measure.
coherence_model = CoherenceModel(topics=nmf_topics, texts=tt_df_sub['transcript'].apply(str.split), dictionary=transcript_dict, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Print the coherence score.
print(f"Coherence Score: {coherence_score}")

2023-11-20 14:36:28,917 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-11-20 14:36:31,395 : INFO : built Dictionary<73015 unique tokens: ['20', '238', '28second', '30second', '31']...> from 4005 documents (total 3494475 corpus positions)
2023-11-20 14:36:31,410 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<73015 unique tokens: ['20', '238', '28second', '30second', '31']...> from 4005 documents (total 3494475 corpus positions)", 'datetime': '2023-11-20T14:36:31.410912', 'gensim': '4.2.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-11-20 14:36:31,795 : INFO : using ParallelWordOccurrenceAccumulator<processes=11, batch_size=64> to estimate probabilities from sliding windows
2023-11-20 14:36:43,082 : INFO : 1 batches submitted to accumulate stats from 64 documents (83903 virtual)
2023-11-20 14:36:43,093 : INFO : 2 batches submitted to accumulate stats fr

Coherence Score: 0.6848976765913517


#### CTM 

In [ ]:
# Prepare the data for CTM.
tp = TopicModelDataPreparation("bert-base-nli-mean-tokens")

training_dataset = tp.fit(text_for_contextual=tt_df['transcript'].tolist(), 
                          text_for_bow=tt_df_sub['transcript'].tolist())

In [ ]:
# Define the size of the vocabulary and contextual embeddings.
bow_size = len(tp.vocab)  
contextual_size = 768  

num_topics = 10  
num_epochs = 10  

# Initialize the CombinedTM model.
ctm = CombinedTM(bow_size=bow_size, contextual_size=contextual_size, n_components=num_topics, num_epochs=num_epochs)

# Train the model.
ctm.fit(training_dataset)

# Extract and display the topics.
topics = ctm.get_topic_lists(10)
for topic_idx, topic in enumerate(topics):
    print(f"Topic {topic_idx + 1}: {', '.join(topic)}")

In [ ]:
# Convert the CTM topics to a format suitable for coherence calculation.
ctm_topics = ctm.get_topic_lists(10)

# Create a dictionary and corpus for the topics.
dictionary = corpora.Dictionary(ctm_topics)
corpus = [dictionary.doc2bow(topic) for topic in ctm_topics]

# Calculate the coherence score using UMass coherence (you can also use C_v coherence).
coherence_model = CoherenceModel(model=None, corpus=corpus, dictionary=dictionary, coherence='u_mass', topics=ctm_topics)
coherence_score = coherence_model.get_coherence()

print(f"Coherence Score: {coherence_score}")